In [121]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import pickle
import sklearn.multiclass as sm
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [111]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
def normalize(x):
    min_val = np.min(x)
    max_val = np.max(x)
    x = (x-min_val) / (max_val-min_val)
    return x

def convert(data):
    im = data
    im_r = im[0:1024].reshape(32, 32).astype('float64')
    im_g = im[1024:2048].reshape(32, 32).astype('float64')
    im_b = im[2048:].reshape(32, 32).astype('float64')
    img = np.dstack((im_r, im_g, im_b))
#     grey = rgb2gray(img)
    return normalize(data).flatten()

In [112]:
def extract_data():
    train_data=[]
    for i in range(1,6):
        file='./cifar-10-batches-py/data_batch_'+str(i)
        temp=unpickle(file)
        temp_data=[]
        mat_data=[]
        for j in range(len(temp[b'data'])):
            op=convert(temp[b'data'][j])
            mat_data.append(op)
        temp_data.append(np.asarray(mat_data).astype('float64'))
        temp_data.append(np.asarray(temp[b'labels']).reshape(temp[b'data'].shape[0],1))
        train_data.append(temp_data)
    temp=unpickle('./cifar-10-batches-py/test_batch')
    temp_data=[]
    mat_data=[]
    for j in range(len(temp[b'data'])):
        op=convert(temp[b'data'][j])
        mat_data.append(op)
    temp_data.append(np.asarray(mat_data).astype('float64'))
    temp_data.append(np.asarray(temp[b'labels']).reshape(temp[b'data'].shape[0],1))
    test_data=temp_data
    temp=unpickle('./cifar-10-batches-py/batches.meta')
    labels=temp[b'label_names']
    for i in range(len(labels)):
        labels[i]=(labels[i].decode("utf-8"))
    return train_data,test_data,labels



In [127]:
def normalize123(data,test):
    data=data.astype('float64')
    test=test.astype('float64')
    mean = np.mean(data, axis = 0)
    data -= mean
    test -= mean
    return data,test

def roc(ovo,X_test,Y_test):
    probabilities = ovo.predict_proba(X_test)
    y_proba = probabilities[:, 1]
    false_positive_rate, true_positive_rate, thresholds = roc_curve(Y_test, y_proba, pos_label=1)
    roc_auc = auc(false_positive_rate, true_positive_rate)
    plt.title('Receiver Operating Characteristic')
    roc_plot = plt.plot(false_positive_rate,
                        true_positive_rate,
                        label='AUC = {:0.2f}'.format(roc_auc))

    plt.legend(loc=0)
    plt.plot([0,1], [0,1], ls='--')
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    
def svm_linear(data):
    X_train=data[0][0]
    X_test=data[1][0]
    Y_train=data[0][1]
    Y_test=data[1][1]
    svm = SVC(kernel='linear')
    ovo = sm.OneVsOneClassifier(svm)
    ovo.fit(X_train,Y_train.ravel())
    y_pred = ovo.predict(X_test)
    accuracy = accuracy_score(Y_test.ravel(), y_pred)
    print("Model accuracy is: ", accuracy)
    results = confusion_matrix(Y_test.ravel(),y_pred) 
    print("Confusion Matrix: ",results) 
    roc(ovo,X_test,Y_test.ravel())

In [114]:
def pca(train,test):
    pca = PCA(n_components=500, random_state=0, svd_solver='randomized')
    pca.fit(train[0])
    X_train = pca.transform(train[0])
    X_test = pca.transform(test[0])
    return X_train,X_test
    

In [115]:
def pca_proc(i,test_data):
    train,test=pca(i,test_data)
    print(train.shape)
    print(test.shape)
    train=[train,i[1]]
    test=[test,test_data[1]]
    return [train,test]
   

In [125]:
train_data,test_data,labels=extract_data()
batch=[]
for i in train_data:
    temp_test=test_data
    i[0],temp_test[0]=normalize123(i[0],temp_test[0])
    batch.append(pca_proc(i,temp_test))
print(len(batch))

(10000, 500)
(10000, 500)
(10000, 500)
(10000, 500)
(10000, 500)
(10000, 500)
(10000, 500)
(10000, 500)
(10000, 500)
(10000, 500)
5


In [128]:
svm_linear(batch[0])

Model accuracy is:  0.3396
Confusion Matrix:  [[368  39  84  50  39  44  23  55 218  80]
 [ 53 382  48  61  36  58  44  46  99 173]
 [100  34 247  89 136  93 135  78  50  38]
 [ 36  59 100 235  87 171 148  61  58  45]
 [ 41  28 165  84 265  81 160 116  33  27]
 [ 32  54 113 158 125 260 105  65  59  29]
 [ 16  44  93 137 111  82 396  55  30  36]
 [ 51  50  85  75 113  99  52 370  38  67]
 [160  74  39  42  18  33  20  21 505  88]
 [ 74 186  30  54  42  22  38  72 114 368]]


AttributeError: 'OneVsOneClassifier' object has no attribute 'predict_proba'